EL PROBLEMA TRATA DE REABASTECER A SURTIDORES:

-TENDREMOS 5 SURTIDORES LLAMADOS 1, 2, 3, 4 Y 5
-DOS DIFERENTES TIPOS DE CISTERNAS, LAS CUALES SERAN NUESTROS NODOS DE SUMINISTRO, LLAMADAS A Y B
-CADA SURTIDOR TIENE DIFERENTES DEMANDAS DE COMBUSTIBLE Y LLEVAR A CADA SURTIDOR TIENE DIFERENTE COSTO DEBIDO A LA DISTANCIA EN LA QUE SE ENCUENTRA

Importar las funciones de PULP

In [353]:
from pulp import *

Crear una lista de todos los nodos de suministro

In [354]:
Almacenes = ["A", "B"]

Cantidad de unidades que distribuye cada nodo de suministro

In [355]:
Nodo_suministro = {"A": 1000,
                   "B": 4000}

Crear lista de todos los nodos a los que se distribuirá

In [356]:
Surtidores = ["1", "2", "3", "4", "5"]

Cantidad que demanda cada nodo

In [357]:
Demanda = {"1":500,
          "2":900,
          "3":1800,
          "4":200,
          "5":700,}

Dinero que cuesta transportar cada litro de combustible en cada ruta de transporte

In [358]:
Costos = [#Surtidores
         #1 2 3 4 5
         [2,4,5,2,1],#Almacen A   
         [3,1,3,2,3] #Almacen B
         ]

Costos = makeDict([Almacenes,Surtidores],Costos,0)

Crear una variable que contenga los datos del problema

In [359]:
prob = LpProblem("Mejor ruta para distribuir y abastecer combustible",LpMinimize)

Crear una lista de todas las posibles rutas para el transporte

In [360]:
Rutas = [(w,b) for w in Almacenes for b in Surtidores]

Crear una variable que contenga las rutas

In [361]:
aux = LpVariable.dicts("Ruta",(Almacenes,Surtidores),0,None,LpInteger)

Se añadira el calculo del costo total del transporte a la variable que contiene los datos del problema

In [362]:
prob += lpSum([aux[w][b]*Costos[w][b] for (w,b) in Rutas])

Los datos de las cantidades maximas que se pueden suministrar se agregan a la variable del problema para cada nodo de suministro

In [363]:
for w in Almacenes:
    prob += lpSum([aux[w][b] for b in Surtidores])<=Nodo_suministro[w]

Configuración y simulación del problema

In [364]:
# Los datos de las cantidades maximas que se pueden suministrar se agregan a la variable del problema 
# para cada nodo de suministro
restriccion_surtidor = {}
for b in Surtidores:
    restriccion = lpSum([aux[w][b] for w in Almacenes])>=Demanda[b]
    prob += restriccion
    restriccion_surtidor[b] = restriccion

#Se tomara rangos de ejemplo
for Demanda in range(500, 601, 200):
    # Reoptimizar el problema aumentando la demanda en el surtidor '1'
    restriccion_surtidor['1'].constant = - Demanda
    prob.solve()
    # Se imprimirá el estado de la solución y cada ruta con su valor óptimo
    print("Estado:", LpStatus[prob.status])
    for v in prob.variables():
        print(v.name, "=", v.varValue)
    # Se imprimirá el costo total del transporte
    print("Costo total del transporte = ", value(prob.objective))

Estado: Optimal
Ruta_A_1 = 300.0
Ruta_A_2 = 0.0
Ruta_A_3 = 0.0
Ruta_A_4 = 0.0
Ruta_A_5 = 700.0
Ruta_B_1 = 200.0
Ruta_B_2 = 900.0
Ruta_B_3 = 1800.0
Ruta_B_4 = 200.0
Ruta_B_5 = 0.0
Costo total del transporte =  8600.0
